In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Course Project: An Arbitrage-Free Smile Interpolator\n",
    "Project report should be submitted as a Jupyter notebook (.ipynb). The report should be written based on this notebook. Please make your formulas and code, test results, observations, intermediate results, improvements made/suggested (optional), and conclusions clear.\n",
    "\n",
    "#### Objectives\n",
    "  - Implement an arbitrage free smile interpolator SmileAF.\n",
    "  - Use the arbitrage free smile interpolator to construct local volatility model\n",
    "  - Use PDE with local volatility model to price a given set of European options (strike in delta $\\times$ maturity)\n",
    "  - Compare the price errors of arbitrage-free smile interpolator and the cubic spline smile interpolator\n",
    "\n",
    "##### Smile Arbitrage\n",
    "1. European call prices are monotonically decreasing with respect to the strike:\n",
    "\n",
    "    \\begin{align}\n",
    "    C(S_0, K_1, T, \\sigma(K_1), r, q) \\geq C(S_0, K_2, T, \\sigma(K_2), r, q) ~\\text{for}~K_1 < K_2\n",
    "    \\end{align}\n",
    "\n",
    "\n",
    "2. The European call price as a function of strike has to be convex every where: for any three points $K_1 < K_2 < K_3$\n",
    "\n",
    "    \\begin{align}\n",
    "    \\frac{C(K_2) - C(K_1) } {K_2 - K_1} < \\frac{C(K_3) - C(K_2) } {K_3 - K_2}\n",
    "    \\end{align}\n",
    "\n",
    "    or\n",
    "\n",
    "    \\begin{align}\n",
    "    C(K_2)  < C(K_3)\\frac{K_2 - K_1} {K_3 - K_1} + C(K_1)\\frac{K_3-K_2} {K_3 - K_1}\n",
    "    \\end{align}\n",
    "\n",
    "    This is also equivalent to \"butterfly price has to be non-negative\".\n",
    "\n",
    "##### When Could Smile Arbitrage Happen?\n",
    "\n",
    "The undiscounted call price is the expectation of payoff under risk neutral measure\n",
    "\n",
    "\\begin{align}\n",
    "C(K) = E[\\max(S-K, 0)]\n",
    "\\end{align}\n",
    "\n",
    "And expectation is an integral over the probability density function $p(s)$\n",
    "\n",
    "\\begin{align}\n",
    "C(K) = \\int_{K}^{+\\infty} (s-K) p(s) ds\n",
    "\\end{align}\n",
    "\n",
    "The 1st non-arbitrage condition translates to\n",
    "\n",
    "\\begin{align}\n",
    "& C(K_1) - C(K_2) = \\left[ \\int_{K_1}^{K_2} (s-K_1) p(s) ds  + \\int_{K_2}^{+\\infty} (K_2-K_1) p(s) ds \\right]\n",
    "\\end{align}\n",
    "\n",
    "which is positive by definition if $K_2 > K_1$.\n",
    "\n",
    "The 2nd non-arbitrage condition translates to\n",
    "\n",
    "\\begin{align}\n",
    "& C(K_3)\\frac{K_2 - K_1} {K_3 - K_1} + C(K_1)\\frac{K_3-K_2} {K_3 - K_1} - C(K_2) \\\\\n",
    "= & \\frac{K_3-K_2} {K_3 - K_1} \\int_{K_1}^{K_2} (s-K_1) p(s) ds + \\frac{K_2 - K_1} {K_3 - K_1} \\int_{K_2}^{K_3} (K_3 - s) p(s) ds\n",
    "\\end{align}\n",
    "\n",
    "which is also positive by definition if $K_3 > K_2 > K_1$.\n",
    "\n",
    "So, when could smile arbitrage happen? When the probability density does not exist. If we can start with valid probability density function $p(s)$, arbitrage-freeness is guaranteed by construction.\n",
    "\n",
    "\n",
    "#### Arbitrage Free Smile (Based on [Fengler 2009])\n",
    "\n",
    "- We consider smile construction for a given expiry $T$.\n",
    "- Start with $N$ discrete sample strike points\n",
    "\n",
    "    \\begin{align*}\n",
    "    \\vec{k} = [k_1, k_2, \\ldots, k_{N}]^{\\top}\n",
    "    \\end{align*}\n",
    "\n",
    "- Try to solve for undiscounted call prices for these $N$ sample points\n",
    "\n",
    "    \\begin{align*}\n",
    "    \\vec{c} = [c_1, c_2, \\ldots, c_N]^{\\top}\n",
    "    \\end{align*}\n",
    "\n",
    "- For the undiscounted call price $C(K)$ for any $K$, we can interpolate using cubic spline over the sample points $(k_i, c_i)$. (Note that we are using cubic spline to interpolate the prices, not volatility)\n",
    "\n",
    "- The second derivative of call price with respect to strike is the probability density function:\n",
    "\n",
    "    \\begin{align}\n",
    "    \\frac{d C}{d K} & =  d \\frac{\\int_K^{\\infty} Sp(S) dS}{dK} - d \\frac{K\\int_K^{\\infty} p(S) dS}{dK} = -Kp(K) - \\left( \\int_K^{\\infty} p(S) dS - K p(K)\\right) = -\\int_K^{\\infty} p(S) dS \\\\\n",
    "    \\frac{d^2 C}{d K^2} & = p(K)\n",
    "    \\end{align}\n",
    "\n",
    "  So $c_i''$ is probability density function at $k_i$, we denote it as $p_i$\n",
    "\n",
    "- Second derivatives in cubic spline interpolation form line segments. Cubic spline on $C(K)$, means linearly interpolate on probability density. If $p_i$ are all positive, the whole pdf is positive by construction --- no smile arbitrage.\n",
    "\n",
    "- For tails --- call prices are almost linear if strike is very far away from spot, we can use natural cubic spline: $p_1 = p_N = 0$.\n",
    "\n",
    "- Our problem is to solve for $[c_1, c_2, \\ldots, c_{N}, p_2, \\ldots, p_{N-1}]$\n",
    "\n",
    "#### Inputs to our problem\n",
    "\n",
    "Same as our Cubic Spline smile interpolator, we have the input marks to start with to construct the Arb-Free(AF) smile interpolator:\n",
    "\n",
    " - Marks:  strike to volatility pairs, denote as $(\\hat k_j, \\sigma_j)$, for $j \\in [1, 2, \\ldots, M]$. In our case, $M=5$.\n",
    "\n",
    "We would like to match the marks exactly. And we cannot directly construct a cubic spline using the $M$ points --- too coarse and distribution is not realistic.\n",
    "\n",
    "#### Problem Definition\n",
    "- We use $N = 50$ sample points, ranging from $[k_1 = S e^{(r_d - r_f)T -\\frac12\\sigma_{ATM}^2T - 5 \\sigma_{ATM} \\sqrt{T}}, k_N = S e^{(r_d - r_f)T -\\frac12\\sigma_{ATM}^2T + 5 \\sigma_{ATM} \\sqrt{T} }]$, i.e., $\\pm 5$ standard deviation based on $\\sigma_{ATM}$.\n",
    "- $\\sigma_{ATM}$ is implied volatility of the middle point of the input marks.\n",
    "- We also assume the strike of the middle point of the input marks is the forward --- ATM forward convention.\n",
    "- The sample points are equally spaced, denote the length of the segment $u = \\frac{k_N - k_1}{N-1}$\n",
    "- We would like the call prices to be as smooth as possible --- minimize the change of the slopes\n",
    "- We want to match exactly the $M$ input marks.\n",
    "- This is a constrained optimization problem.\n",
    "\n",
    " Constraints\n",
    " - Cubic spline interpolation imposes the constraints that the left and right first derivative of a point have to match, it can be derived by matching the first  derivative of the left and right segments for point $i$ we have the condition\n",
    "\n",
    "   \\begin{align}\n",
    "   c_{i+1} + c_{i-1} - 2 c_{i} = (\\frac23 p_i + \\frac16 p_{i+1} + \\frac16 p_{i-1}) u^2\n",
    "   \\end{align}\n",
    "\n",
    "   The cubic spline constraints translate to the linear system\n",
    "\n",
    "   \\begin{align} \\underbrace{\\begin{pmatrix}\n",
    "   1 & -2 & 1 & 0 & \\ldots & 0 \\\\\n",
    "   0 & 1 & -2 & 1 & \\ddots & \\vdots \\\\\n",
    "   \\vdots & \\ddots & \\ddots & \\ddots & \\ddots & 0 \\\\\n",
    "   0 & \\ldots & 0 & 1 & -2 & 1\n",
    "   \\end{pmatrix}}{\\vec{Q}{(N-2) \\times N}}\n",
    "   \\begin{pmatrix}\n",
    "   c_1 \\\\\n",
    "   c_2 \\\\\n",
    "   \\vdots \\\\\n",
    "   c_N\n",
    "   \\end{pmatrix} =\n",
    "   \\underbrace{u^2\n",
    "   \\begin{pmatrix}\n",
    "   \\frac23 & \\frac16 & 0 & \\ldots & 0 \\\\\n",
    "   \\frac16 & \\frac23 & \\frac16 & \\ddots & \\vdots \\\\\n",
    "   0 & \\ddots & \\ddots & \\ddots & 0 \\\\\n",
    "   \\vdots & \\ddots & \\frac 1 6 & \\frac23 & \\frac16 \\\\\n",
    "   0 & \\ldots & 0 & \\frac 1 6 &  \\frac23\n",
    "   \\end{pmatrix}}{\\vec{R}{(N-2) \\times (N-2)}}\n",
    "   \\begin{pmatrix}\n",
    "   p_2 \\\\\n",
    "   p_3 \\\\\n",
    "   \\vdots \\\\\n",
    "   p_{N-1}\n",
    "   \\end{pmatrix}\n",
    "   \\end{align}\n",
    "\n",
    "   If we define\n",
    "   \\begin{align}\n",
    "   \\vec{x} =\n",
    "   \\begin{pmatrix}\n",
    "   \\vec{c}^{\\top} \\\\\n",
    "   \\vec{p}^{\\top} \\\\\n",
    "   \\end{pmatrix}, ~\n",
    "   \\vec{A} = (\\vec{Q}, -\\vec{R})\n",
    "   \\end{align}\n",
    "\n",
    "   we can represent the constraint as:\n",
    "   \\begin{align}\n",
    "   \\vec{Ax} = \\vec{0}        ~\\textbf{--- Constraint 1}\n",
    "   \\end{align}\n",
    "\n",
    " - The call prices at the input marks $\\hat k_j, j \\in [1, 2, \\ldots, M]$ can be represented by cubic spline interpolation\n",
    "   \\begin{align}\n",
    "   C(\\hat k_j) =  a c_i + b c_{i+1} + \\frac{(a^3 - a)u^2}6 p_i + \\frac{(b^3-b) u^2}6 p_{i+1}\n",
    "   ~\\textbf{--- Constraint 2}\n",
    "   \\end{align}\n",
    "\n",
    "   where\n",
    "\n",
    "   \\begin{align}\n",
    "   a = \\frac{k_{i+1} - \\hat k_j}{u},~~~b = 1-a\n",
    "   \\end{align}\n",
    "\n",
    "   and $[k_i, k_{i+1}]$ here represents the segment that $\\hat k_j$ falls in.\n",
    "\n",
    " - $p_i$ are densities, so\n",
    "\n",
    "    \\begin{align}\n",
    "    p_i > 0 ~\\textbf{--- Constraint 3}\n",
    "    \\end{align}\n",
    "\n",
    " - Integrating the density function we should get 1.0 (recall that density function are linearly interpolated)\n",
    "\n",
    "    \\begin{align}\n",
    "    u \\sum p_i = 1.0 ~\\textbf{--- Constraint 4}\n",
    "    \\end{align}\n",
    "\n",
    " - Natural cubic spline, $p_1$ and $p_N$ are zero, so we could solve directly $c_1$ and $c_N$\n",
    "\n",
    "    \\begin{align}\n",
    "    c_1 = Se^{(r_d - r_f)T} - k_1, ~c_N = 0  ~\\textbf{--- Constraint 5}\n",
    "    \\end{align}\n",
    "\n",
    " - Call prices are monotonically decreasing:\n",
    "\n",
    "    \\begin{align}\n",
    "    c_{i+1} - c_{i} \\leq 0 \\text{for}i \\in \\{1, 2, \\ldots, N-1\\}  ~\\textbf{--- Constraint 6}\n",
    "    \\end{align}\n",
    "\n",
    "\n",
    " Objective Function\n",
    "\n",
    " - Fill the rest of the DOF using objective function (soft constraints)\n",
    "\n",
    " - [Fengler 2009] tried minimizing the below to achieve smoothness on $p$:\n",
    "\n",
    "    \\begin{align}\n",
    "    \\int_{k_1}^{k_N} p(S)^2 dS = \\text{constant} \\times \\vec{p}^{\\top} \\vec{R} \\vec{p}\n",
    "    \\end{align}\n",
    "\n",
    "    Using $\\vec{x}$ as variable and define\n",
    "\n",
    "    \\begin{align}\n",
    "    \\vec{H}_{(2N-2) \\times (2N-2)} =\n",
    "    \\begin{pmatrix}\n",
    "    \\vec{0} & \\vec{0} \\\\\n",
    "    \\vec{0} & \\vec{R}_{(N-2) \\times (N-2)}\n",
    "    \\end{pmatrix}\n",
    "    \\end{align}\n",
    "\n",
    "    the problem becomes minimizing\n",
    "\n",
    "    \\begin{align}\n",
    "    \\vec{x}^\\top \\vec{H} \\vec{x}\n",
    "    \\end{align}\n",
    "\n",
    "Problem Formulation\n",
    "\n",
    " We can formulate our problem as\n",
    "    \\begin{align}\n",
    "    \\min~~~\\vec{x}^\\top \\vec{H} \\vec{x}\n",
    "    \\end{align}\n",
    " subject to constraints 1 to 5.\n",
    " - All the constraints are linear function of $\\vec{x}$\n",
    " - Our objective function is quadratic and the matrix $\\vec{H}$ is positive semi-definite\n",
    " - Global solution exists, and (relatively) efficient to solve\n",
    "\n",
    "Tips\n",
    " - To solve the quadratic programming problem, we can use the CVXOPT package:\n",
    "   - http://cvxopt.org/examples/tutorial/qp.html\n",
    "   - https://courses.csail.mit.edu/6.867/wiki/images/a/a7/Qp-cvxopt.pdf\n",
    " - Write down the exact formulas using the same symbols used by CVXOPT QP problem's documentation in the above docs, then translate them into code. This will make debugging easier.\n",
    " - To check whether solver's result makes sense, examine if the constraints are satisified, and if the call prices are smooth and match the input.\n",
    " - If test run takes too long, reduce the number of grid points in PDE pricer, or skip the calibration report and inspect the volatility surface first.\n",
    " - It might be easier to plot implied vol, call prices, PDF, and the marks to check the result.\n",
    " - use bisect.bisect_left to find the bucket $\\hat{k}$ belongs to (https://docs.python.org/3/library/bisect.html)\n",
    "\n",
    "References\n",
    "\n",
    "[Fengler 2009] Arbitrage-free smoothing of the implied volatility surface, Quantitative Finance, 2009\n",
    "\n",
    "*\n",
    "### Implementation"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "pycharm": {
     "name": "#%% md\n"
    }
   },
   "source": [
    "Below are some building blocks for the project. You contribution should be in SmileAF class.\n",
    "You can modify any other classes or methods. If you do so, please describe your modification in the project report."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "import math\n",
    "from enum import Enum\n",
    "from scipy import optimize\n",
    "import bisect\n",
    "from scipy.interpolate import CubicSpline\n",
    "\n",
    "class PayoffType(Enum):\n",
    "    Call = 0\n",
    "    Put = 1\n",
    "\n",
    "def cnorm(x):\n",
    "    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0\n",
    "\n",
    "def fwdDelta(fwd, stdev, strike, payoffType):\n",
    "    d1 = math.log(fwd / strike) / stdev + stdev / 2\n",
    "    if payoffType == PayoffType.Call:\n",
    "        return cnorm(d1)\n",
    "    elif payoffType == PayoffType.Put:\n",
    "        return - cnorm(-d1)\n",
    "    else:\n",
    "        raise Exception(\"not supported payoff type\", payoffType)\n",
    "\n",
    "# solve for the K such that Delta(S, T, K, vol) = delta\n",
    "def strikeFromDelta(S, r, q, T, vol, delta, payoffType):\n",
    "    fwd = S * math.exp((r-q) * T)\n",
    "    if payoffType == PayoffType.Put:\n",
    "        delta = -delta\n",
    "    f = lambda K: (fwdDelta(fwd, vol * math.sqrt(T), K, payoffType) - delta)\n",
    "    a, b = 0.0001, 10000\n",
    "    return optimize.brentq(f, a, b)\n",
    "\n",
    "class ImpliedVol:\n",
    "    def init(self, ts, smiles):\n",
    "        self.ts = ts\n",
    "        self.smiles = smiles\n",
    "    # linear interpolation in variance, along the strike line\n",
    "    def Vol(self, t, k):\n",
    "        # locate the interval t is in\n",
    "        pos = bisect.bisect_left(self.ts, t)\n",
    "        # if t is on or in front of first pillar,\n",
    "        if pos == 0:\n",
    "            return self.smiles[0].Vol(k)\n",
    "        if pos >= len(self.ts):\n",
    "            return self.smiles[-1].Vol(k)\n",
    "        else:  # in between two brackets\n",
    "            prevVol, prevT = self.smiles[pos-1].Vol(k), self.ts[pos-1]\n",
    "            nextVol, nextT = self.smiles[pos].Vol(k), self.ts[pos]\n",
    "            w = (nextT - t) / (nextT - prevT)\n",
    "            prevVar = prevVol * prevVol * prevT\n",
    "            nextVar = nextVol * nextVol * nextT\n",
    "            return  math.sqrt((w * prevVar + (1-w) * nextVar)/t)\n",
    "        return\n",
    "\n",
    "    def dVoldK(self, t, k):\n",
    "        return (self.Vol(t, k+0.001) - self.Vol(t, k-0.001)) / 0.002\n",
    "    def dVoldT(self, t, k):\n",
    "        return (self.Vol(t+0.005, k) - self.Vol(t, k)) / 0.005\n",
    "    def dVol2dK2(self, t, k):\n",
    "        return (self.Vol(t, k+0.001) + self.Vol(t, k-0.001) - 2*self.Vol(t, k)) / 0.000001\n",
    "\n",
    "class LocalVol:\n",
    "    def init(self, iv, S0, rd, rf):\n",
    "        self.iv = iv\n",
    "        self.S0 = S0\n",
    "        self.rd = rd\n",
    "        self.rf = rf\n",
    "    def LV(self, t, s):\n",
    "        if t < 1e-6:\n",
    "            return self.iv.Vol(t, s)\n",
    "        imp = self.iv.Vol(t, s)\n",
    "        dvdk = self.iv.dVoldK(t, s)\n",
    "        dvdt = self.iv.dVoldT(t, s)\n",
    "        d2vdk2 = self.iv.dVol2dK2(t, s)\n",
    "        d1 = (math.log(self.S0/s) + (self.rd-self.rf)*t + imp * imp * t / 2) / imp / math.sqrt(t)\n",
    "        numerator = imp*imp + 2*t*imp*dvdt + 2*(self.rd-self.rf)*s*t*imp*dvdk\n",
    "        denominator = (1+s*d1*math.sqrt(t)dvdk)2 + s*s*t*imp(d2vdk2 - d1 * math.sqrt(t) * dvdk * dvdk)\n",
    "        localvar = min(max(numerator / denominator, 1e-8), 1.0)\n",
    "        if numerator < 0: # floor local volatility\n",
    "            localvar = 1e-8\n",
    "        if denominator < 0: # cap local volatility\n",
    "            localvar = 1.0\n",
    "        return math.sqrt(localvar)\n",
    "\n",
    "class EuropeanOption():\n",
    "    def init(self, assetName, expiry, strike, payoffType):\n",
    "        self.assetName = assetName\n",
    "        self.expiry = expiry\n",
    "        self.strike = strike\n",
    "        self.payoffType = payoffType\n",
    "    def payoff(self, S):\n",
    "        if self.payoffType == PayoffType.Call:\n",
    "            return max(S - self.strike, 0)\n",
    "        elif self.payoffType == PayoffType.Put:\n",
    "            return max(self.strike - S, 0)\n",
    "        else:\n",
    "            raise Exception(\"payoffType not supported: \", self.payoffType)\n",
    "    def valueAtNode(self, t, S, continuation):\n",
    "        if continuation == None:\n",
    "            return self.payoff(S)\n",
    "        else:\n",
    "            return continuation\n",
    "\n",
    "# Black-Scholes analytic pricer\n",
    "def bsPrice(S, r, q, vol, T, strike, payoffType):\n",
    "    fwd = S * math.exp((r-q) * T)\n",
    "    stdev = vol * math.sqrt(T)\n",
    "    d1 = math.log(fwd / strike) / stdev + stdev / 2\n",
    "    d2 = d1 - stdev\n",
    "    if payoffType == PayoffType.Call:\n",
    "        return math.exp(-r * T) * (fwd * cnorm(d1) - cnorm(d2) * strike)\n",
    "    elif payoffType == PayoffType.Put:\n",
    "        return math.exp(-r * T) * (strike * cnorm(-d2) - cnorm(-d1) * fwd)\n",
    "    else:\n",
    "        raise Exception(\"not supported payoff type\", payoffType)\n",
    "\n",
    "# PDE pricer with local volatility\n",
    "def pdePricerX(S0, r, q, lv, NX, NT, w, trade):\n",
    "    # set up pde grid\n",
    "    mu = r - q\n",
    "    T = trade.expiry\n",
    "    X0 = math.log(S0)\n",
    "    vol0 = lv.LV(0, S0)\n",
    "    srange = 5 * vol0 * math.sqrt(T)\n",
    "    maxX = X0 + (mu - vol0 * vol0 * 0.5)*T + srange\n",
    "    minX = X0 - (mu - vol0 * vol0 * 0.5)*T - srange\n",
    "    dt = T / (NT-1)\n",
    "    dx = (maxX - minX) / (NX-1)\n",
    "    # set up spot grid\n",
    "    xGrid = np.array([minX + i*dx for i in range(NX)])\n",
    "    # initialize the payoff\n",
    "    ps = np.array([trade.payoff(math.exp(x)) for x in xGrid])\n",
    "    # backward induction\n",
    "    for j in range(1, NT):\n",
    "        # set up the matrix, for LV we need to update it for each iteration\n",
    "        M = np.zeros((NX, NX))\n",
    "        D = np.zeros((NX, NX))\n",
    "        for i in range(1, NX - 1):\n",
    "            vol = lv.LV(j*dt, math.exp(xGrid[i]))\n",
    "            M[i, i - 1] = (mu - vol * vol / 2.0) / 2.0 / dx - vol * vol / 2 / dx / dx\n",
    "            M[i, i] = r + vol * vol / dx / dx\n",
    "            M[i, i + 1] = -(mu - vol * vol / 2.0) / 2.0 / dx - vol * vol / 2 / dx / dx\n",
    "            D[i, i] = 1.0\n",
    "        # the first row and last row depends on the boundary condition\n",
    "        M[0, 0], M[NX - 1, NX - 1] = 1.0, 1.0\n",
    "        rhsM = (D - dt * M) * w + (1 - w) * np.identity(NX)\n",
    "        lhsM = w * np.identity(NX) + (D + dt * M) * (1 - w)\n",
    "        inv = np.linalg.inv(lhsM)\n",
    "\n",
    "        ps = rhsM.dot(ps)\n",
    "        ps[0] = dt*math.exp(-r*j*dt) * trade.payoff(math.exp(xGrid[0])) # discounted payoff\n",
    "        ps[NX-1] = dt*math.exp(-r*j*dt) * trade.payoff(math.exp(xGrid[NX-1]))\n",
    "        ps = inv.dot(ps)\n",
    "    # linear interpolate the price at S0\n",
    "    return np.interp(X0, xGrid, ps)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "pycharm": {
     "name": "#%% md\n"
    }
   },
   "source": [
    "Below are the smile interpolators and smile constructor. You need to implement SmileAF. Note that smileFromMarks takes a parameter smileInterpMethod. When it is 'AF', SmileAF is used."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {
    "pycharm": {
     "name": "#%%\n"
    }
   },
   "outputs": [],
   "source": [
    "class SmileAF:\n",
    "    def init(self, strikes, vols, T):\n",
    "        self.atmvol = vols[int(len(vols)/2)]\n",
    "        self.fwd = strikes[int(len(strikes)/2)]\n",
    "        self.T = T\n",
    "        self.N = 50\n",
    "        stdev = self.atmvol * math.sqrt(T)\n",
    "        kmin = self.fwd * math.exp(-0.5*stdev*stdev-5 * stdev)\n",
    "        kmax = self.fwd * math.exp(-0.5*stdev*stdev+5 * stdev)\n",
    "        u = (kmax - kmin) / (self.N - 1)\n",
    "        self.ks = [kmin + u * i for i in range(0, self.N)]\n",
    "        self.cs = np.zeros(self.N)  # undiscounted call option prices\n",
    "        self.ps = np.zeros(self.N)  # densities\n",
    "        self.u = u\n",
    "        # now we need to construct our constrained optimization problem to solve for cs and ps\n",
    "        # ... YOUR CODE HERE ... to solve for self.cs and self.ps\n",
    "        # ...\n",
    "\n",
    "        # now we obtained cs and ps, we do not interpolate for price for any k and imply the vol,\n",
    "        # since at the tails the price to vol gradient is too low and is numerically not stable.\n",
    "        # Instead, we imply the volatilities for all points between put 10 delta and call 10 delta input points\n",
    "        # then we make the vol flat at the wings by setting the vols at kmin and kmax,\n",
    "        # we then construct a cubic spline interpolator on the dense set of volatilities so that it's C2\n",
    "        # and faster then implying volatilities on the fly.\n",
    "        # note that this treatment of tail is simplified. It could also introduce arbitrage.\n",
    "        # In practice, the tails should be calibrated to a certain distribution.\n",
    "        def implyVol(k, prc, v):\n",
    "            stdev = v * math.sqrt(self.T)\n",
    "            d1 = (math.log(self.fwd / k)) / stdev + 0.5 * stdev\n",
    "            d2 = (math.log(self.fwd / k)) / stdev - 0.5 * stdev\n",
    "            return self.fwd * cnorm(d1) - k * cnorm(d2) - prc\n",
    "        khmin = bisect.bisect_left(self.ks, strikes[0])\n",
    "        khmax = bisect.bisect_right(self.ks, strikes[len(strikes)-1])\n",
    "        kks = [0] * ((khmax+1) - (khmin-1) + 2)\n",
    "        vs = [0] * ((khmax+1) - (khmin-1) + 2)\n",
    "        for i in range(khmin-1, khmax+1):\n",
    "            prc = self.Price(self.ks[i])\n",
    "            f = lambda v: implyVol(self.ks[i], prc, v)\n",
    "            a, b = 1e-8, 10\n",
    "            vs[i - (khmin-1) + 1] = optimize.brentq(f, a, b)\n",
    "            kks[i - (khmin-1) + 1] = self.ks[i]\n",
    "        kks[0] = kmin\n",
    "        vs[0] = vs[1]\n",
    "        kks[len(kks)-1] = kmax\n",
    "        vs[len(vs)-1] = vs[len(vs)-2]\n",
    "\n",
    "        self.vs = vs\n",
    "        self.cubicVol = CubicSpline(kks, vs, bc_type=((1, 0.0), (1, 0.0)), extrapolate=True)\n",
    "\n",
    "    def Vol(self, k):\n",
    "        if k < self.ks[0]:  # scipy cubicspline bc_type confusing, extrapolate by ourselfs\n",
    "            return self.vs[0]\n",
    "        if k > self.ks[-1]:\n",
    "            return self.vs[-1]\n",
    "        else:\n",
    "            return self.cubicVol(k)\n",
    "\n",
    "    # undiscounted call price - given cs and ps,\n",
    "    # we can obtain undiscounted call price for any k via cubic spline interpolation\n",
    "    def Price(self, k):\n",
    "        if k <= self.ks[0]:\n",
    "            return self.fwd - k\n",
    "        if k >= self.ks[self.N-1]:\n",
    "            return 0.0\n",
    "        pos = bisect.bisect_left(self.ks, k)\n",
    "        a = (self.ks[pos] - k) / self.u\n",
    "        b = 1 - a\n",
    "        c = (a * a * a - a) * self.u * self.u / 6.0\n",
    "        d = (b * b * b - b) * self.u * self.u / 6.0\n",
    "        return a * self.cs[pos-1] + b * self.cs[pos] + c*self.ps[pos-1] + d*self.ps[pos]\n",
    "        \n",
    "class SmileCubicSpline:\n",
    "    def init(self, strikes, vols):\n",
    "        # add additional point on the right to avoid arbitrage\n",
    "        self.strikes = strikes + [1.1 * strikes[-1] - 0.1 * strikes[-2]]\n",
    "        self.vols = vols + [vols[-1] + (vols[-1] - vols[-2]) / 10]\n",
    "        self.cs = CubicSpline(strikes, vols, bc_type=((1, 0.0), (1, 0.0)), extrapolate=True)\n",
    "\n",
    "    def Vol(self, k):\n",
    "        if k < self.strikes[0]:  # scipy cubicspline bc_type confusing, extrapolate by ourselfs\n",
    "            return self.vols[0]\n",
    "        if k > self.strikes[-1]:\n",
    "            return self.vols[-1]\n",
    "        else:\n",
    "            return self.cs(k)\n",
    "\n",
    "def smileFromMarks(T, S, r, q, atmvol, bf25, rr25, bf10, rr10, smileInterpMethod):\n",
    "    c25 = bf25 + atmvol + rr25/2\n",
    "    p25 = bf25 + atmvol - rr25/2\n",
    "    c10 = bf10 + atmvol + rr10/2\n",
    "    p10 = bf10 + atmvol - rr10/2\n",
    "\n",
    "    ks = [ strikeFromDelta(S, r, q, T, p10, 0.1, PayoffType.Put),\n",
    "           strikeFromDelta(S, r, q, T, p25, 0.25, PayoffType.Put),\n",
    "           S * math.exp((r-q)*T),\n",
    "           strikeFromDelta(S, r, q, T, c25, 0.25, PayoffType.Call),\n",
    "           strikeFromDelta(S, r, q, T, c10, 0.1, PayoffType.Call) ]\n",
    "    # print(T, ks)\n",
    "    if smileInterpMethod == \"CUBICSPLINE\":\n",
    "        return SmileCubicSpline(ks, [p10, p25, atmvol, c25, c10])\n",
    "    elif smileInterpMethod == \"AF\":\n",
    "        return SmileAF(ks, [p10, p25, atmvol, c25, c10], T)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "pycharm": {
     "name": "#%% md\n"
    }
   },
   "source": [
    "Below is a calibration report that shows the calibration error of local volatility PDE pricer."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "import matplotlib.pyplot as plt\n",
    "import numpy as np\n",
    "\n",
    "def createTestFlatVol(S, r, q, smileInterpMethod):\n",
    "    pillars = [0.02, 0.04, 0.06, 0.08, 0.16, 0.25, 0.75, 1.0, 1.5, 2, 3, 5]\n",
    "    atmvols = [0.155, 0.1395, 0.1304, 0.1280, 0.1230, 0.1230, 0.1265, 0.1290, 0.1313, 0.1318, 0.1313, 0.1305, 0.1295]\n",
    "    bf25s = np.zeros(len(atmvols))\n",
    "    rr25s = np.zeros(len(atmvols))\n",
    "    bf10s = np.zeros(len(atmvols))\n",
    "    rr10s = np.zeros(len(atmvols))\n",
    "    smiles = [smileFromMarks(pillars[i], S, r, q, atmvols[i], bf25s[i], rr25s[i], bf10s[i], rr10s[i], smileInterpMethod) for i in range(len(pillars))]\n",
    "    return ImpliedVol(pillars, smiles)\n",
    "\n",
    "def createTestImpliedVol(S, r, q, sc, smileInterpMethod):\n",
    "    pillars = [0.02, 0.04, 0.06, 0.08, 0.16, 0.25, 0.75, 1.0, 1.5, 2, 3, 5]\n",
    "    atmvols = [0.155, 0.1395, 0.1304, 0.1280, 0.1230, 0.1230, 0.1265, 0.1290, 0.1313, 0.1318, 0.1313, 0.1305, 0.1295]\n",
    "    bf25s = [0.0016, 0.0016, 0.0021, 0.0028, 0.0034, 0.0043, 0.0055, 0.0058, 0.0060, 0.0055, 0.0054, 0.0050, 0.0045, 0.0043]\n",
    "    rr25s = [-0.0065, -0.0110, -0.0143, -0.0180, -0.0238, -0.0288, -0.0331, -0.0344, -0.0349, -0.0340, -0.0335, -0.0330, -0.0330]\n",
    "    bf10s = [0.0050, 0.0050, 0.0067, 0.0088, 0.0111, 0.0144, 0.0190, 0.0201, 0.0204, 0.0190, 0.0186, 0.0172, 0.0155, 0.0148]\n",
    "    rr10s = [-0.0111, -0.0187, -0.0248, -0.0315, -0.0439, -0.0518, -0.0627, -0.0652, -0.0662, -0.0646, -0.0636, -0.0627, -0.0627]\n",
    "    smiles = [smileFromMarks(pillars[i], S, r, q, atmvols[i], bf25s[i]*sc, rr25s[i]*sc, bf10s[i]*sc, rr10s[i]*sc, smileInterpMethod) for i in range(len(pillars))]\n",
    "    return ImpliedVol(pillars, smiles)\n",
    "\n",
    "def plotTestImpliedVolSurface(iv):\n",
    "    tStart, tEnd = 0.02, 5\n",
    "    ts = np.arange(tStart, tEnd, 0.1)\n",
    "    fwdEnd = S*math.exp((r-q)*tEnd)\n",
    "    kmin = strikeFromDelta(S, r, q, tEnd, iv.Vol(tEnd, fwdEnd), 0.1, PayoffType.Put)\n",
    "    kmax = strikeFromDelta(S, r, q, tEnd, iv.Vol(tEnd, fwdEnd), 0.1, PayoffType.Call)\n",
    "    ks = np.arange(kmin, kmax, 0.01)\n",
    "    vs = np.ndarray((len(ts), len(ks)))\n",
    "    lv = LocalVol(iv, S, r, q)\n",
    "    lvs = np.ndarray((len(ts), len(ks)))\n",
    "    for i in range(len(ts)):\n",
    "        for j in range(len(ks)):\n",
    "            vs[i, j] = iv.Vol(ts[i], ks[j])\n",
    "            lvs[i, j] = lv.LV(ts[i], ks[j])\n",
    "    hf = plt.figure(figsize=(8, 6), dpi=80)\n",
    "    ha = hf.add_subplot(121, projection='3d')\n",
    "    hb = hf.add_subplot(122, projection='3d')\n",
    "    X, Y = np.meshgrid(ks, ts)\n",
    "    ha.plot_surface(X, Y, vs)\n",
    "    ha.set_title(\"implied vol\")\n",
    "    ha.set_xlabel(\"strike\")\n",
    "    ha.set_ylabel(\"T\")\n",
    "    hb.plot_surface(X, Y, lvs)\n",
    "    hb.set_title(\"local vol\")\n",
    "    hb.set_xlabel(\"strike\")\n",
    "    hb.set_ylabel(\"T\")\n",
    "    plt.show()\n",
    "\n",
    "# the PDE calibration error report takes a implied volatility surface,\n",
    "# verifies the pricing error of the pde pricer with local volatility surface\n",
    "def pdeCalibReport(S0, r, q, impliedVol):\n",
    "    ts = [0.02, 0.04, 0.06, 1/12.0, 1/6.0, 1/4.0, 1/2.0, 1, 2, 5]\n",
    "    ds = np.arange(0.1, 1.0, 0.1)\n",
    "    # ds = np.arange(0.5, 1.7, 0.1)\n",
    "    err = np.zeros((len(ds), len(ts)))\n",
    "    fig, ax = plt.subplots()\n",
    "\n",
    "    ax.set_xticks(np.arange(len(ts)))\n",
    "    ax.set_xlabel(\"T\")\n",
    "    ax.set_yticks(np.arange(len(ds)))\n",
    "    ax.set_ylabel(\"Put Delta\")\n",
    "    ax.set_xticklabels(map(lambda t : round(t, 2), ts))\n",
    "    ax.set_yticklabels(map(lambda d : round(d, 1), ds))\n",
    "\n",
    "    # create local vol surface\n",
    "    lv = LocalVol(impliedVol, S0, r, q)\n",
    "    # Loop over data dimensions and create text annotations.\n",
    "    for i in range(len(ds)):\n",
    "        for j in range(len(ts)):\n",
    "            T = ts[j]\n",
    "            K = strikeFromDelta(S0, r, 0, T, iv.Vol(T, S0*math.exp(r*T)), ds[i], PayoffType.Put)\n",
    "            payoff = PayoffType.Put\n",
    "            trade = EuropeanOption(\"ASSET1\", T, K, payoff)\n",
    "            vol = impliedVol.Vol(ts[j], K)\n",
    "            bs = bsPrice(S0, r, q, vol, T, K, payoff)\n",
    "            pde = pdePricerX(S0, r, q, lv, max(50, int(50 * T)), max(50, int(50 * T)), 0.5, trade)\n",
    "            # normalize error in 1 basis point per 1 unit of stock\n",
    "            err[i, j] = math.fabs(bs - pde)/S0 * 10000\n",
    "            ax.text(j, i, round(err[i, j], 1), ha=\"center\", va=\"center\", color=\"w\")\n",
    "    im = ax.imshow(err)\n",
    "    ax.set_title(\"Dupire Calibration PV Error Matrix\")\n",
    "    fig.tight_layout()\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "pycharm": {
     "name": "#%% md\n"
    }
   },
   "source": [
    "We test with no smile case first. In the calibration error report, we are showing the error in basis points -- 0.01% with respect to 1 notional."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {
    "pycharm": {
     "is_executing": True,
     "name": "#%%\n"
    }
   },
   "outputs": [],
   "source": [
    "S, r, q = 1.25805, 0.01, 0.0\n",
    "iv = createTestImpliedVol(S, r, q, sc = 0.0, smileInterpMethod='CUBICSPLINE')\n",
    "plotTestImpliedVolSurface(iv)\n",
    "pdeCalibReport(S, r, q, iv)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "pycharm": {
     "name": "#%% md\n"
    }
   },
   "source": [
    "Then test smile case with CubicSpline, with a mild smile (tuned by the coeffiicent sc)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {
    "pycharm": {
     "is_executing": True,
     "name": "#%%\n"
    }
   },
   "outputs": [],
   "source": [
    "iv = createTestImpliedVol(S, r, q, sc = 0.5, smileInterpMethod='CUBICSPLINE')\n",
    "plotTestImpliedVolSurface(iv)\n",
    "pdeCalibReport(S, r, q, iv)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Then test smile case with CubicSpline, with a the input smile (sc = 1.0). It can be seen that the short end low strike region has some smile arbitrage. The calibration errors become larger."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {
    "pycharm": {
     "is_executing": True,
     "name": "#%%\n"
    }
   },
   "outputs": [],
   "source": [
    "iv = createTestImpliedVol(S, r, q, sc = 1.0, smileInterpMethod='CUBICSPLINE')\n",
    "plotTestImpliedVolSurface(iv)\n",
    "pdeCalibReport(S, r, q, iv)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "pycharm": {
     "name": "#%% md\n"
    }
   },
   "source": [
    "*\n",
    "### Your test cases with SmileAF"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {
    "pycharm": {
     "is_executing": True,
     "name": "#%%\n"
    }
   },
   "outputs": [],
   "source": [
    "iv = createTestImpliedVol(S, r, q, sc = 1.0, smileInterpMethod='AF')\n",
    "plotTestImpliedVolSurface(iv)\n",
    "pdeCalibReport(S, r, q, iv)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}